In [ ]:
import nltk
from textblob import TextBlob
from newspaper import Article
import tkinter as tk
from tkinter import filedialog
from googletrans import Translator, LANGUAGES
from tkinter import ttk
import pyttsx3

def summarize():
    global title_text, authors_text, publication_date_text, summary_text, sentiment_text
    url = url_entry.get().strip()

    try:
     
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()

        # Extract article details
        title_text = f"Title: {article.title}"
        authors_text = f"Authors: {', '.join(article.authors)}"
        publication_date_text = f"Publication Date: {article.publish_date}"
        summary_text = f"Summary: {article.summary}"

        analysis = TextBlob(article.text)
        polarity = analysis.polarity
        sentiment_text = f"Sentiment: {'positive' if polarity > 0 else 'negative' if polarity < 0 else 'neutral'}"

        # Update GUI with article details
        update_text_widget(title, title_text)
        update_text_widget(author, authors_text)
        update_text_widget(publication, publication_date_text)
        update_text_widget(summary, summary_text)
        update_text_widget(sentiment, sentiment_text)
    except Exception as e:
        update_text_widget(summary, f"Error fetching or processing article: {e}")

def translate():
    target_language = dest_lang.get()
    translator = Translator()

    try:
       
        translated_title = translator.translate(title_text, dest=target_language).text
        translated_authors = translator.translate(authors_text, dest=target_language).text
        translated_publication = translator.translate(publication_date_text, dest=target_language).text
        translated_summary = translator.translate(summary_text, dest=target_language).text
        translated_sentiment = translator.translate(sentiment_text, dest=target_language).text
        
        # Update GUI with translated text
        update_text_widget(title, f"Title: {translated_title}")
        update_text_widget(author, f"Authors: {translated_authors}")
        update_text_widget(publication, f"Publication Date: {translated_publication}")
        update_text_widget(summary, f"Summary: {translated_summary}")
        update_text_widget(sentiment, f"Sentiment: {translated_sentiment}")
    except Exception as e:
        update_text_widget(summary, f"Error translating text: {e}")

def update_text_widget(widget, text):
    widget.config(state='normal')
    widget.delete(1.0, tk.END)
    widget.insert(tk.END, text)
    widget.config(state='disabled')

def savefile():
    file_types = [("Text files", "*.txt")]
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=file_types)

    if file_path:
        try:
            with open(file_path, 'w') as file:
                file.write(title_text + '\n')
                file.write(authors_text + '\n')
                file.write(publication_date_text + '\n')
                file.write(summary_text + '\n')
                file.write(sentiment_text + '\n')
        except Exception as e:
            print(f"Error saving file: {e}")

def copy_to_clipboard():
    root.clipboard_clear()
    root.clipboard_append(summary_text)
    root.update()

def speak():
    try:
        speech = pyttsx3.init()
        voices = speech.getProperty('voices')
        speech.setProperty('voice', voices[0].id) 

        speech.say(title_text)
        speech.say("Authors of the news article are: " + authors_text)
        speech.say("Publication date of the news article is: " + publication_date_text)
        speech.say("Summary of the news article is as follows: " + summary_text)
        speech.say("Sentiment of the news article is: " + sentiment_text)
        speech.runAndWait()
    except Exception as e:
        print(f"Error during text-to-speech: {e}")


nltk.download('punkt')

# Create GUI window
root = tk.Tk()
root.title("News Summarizer")
root.geometry('1400x900')

# Title
tlabel = tk.Label(root, text='Title')
tlabel.pack()
title = tk.Text(root, height=1, width=140)
title.config(state='disabled', bg='lightblue')
title.pack()

# Author
alabel = tk.Label(root, text='Authors')
alabel.pack()
author = tk.Text(root, height=1, width=140)
author.config(state='disabled', bg='lightblue')
author.pack()

# Publication Date
plabel = tk.Label(root, text='Publication Date')
plabel.pack()
publication = tk.Text(root, height=1, width=140)
publication.config(state='disabled', bg='lightblue')
publication.pack()

# Summary
slabel = tk.Label(root, text='Summary')
slabel.pack()
summary = tk.Text(root, height=20, width=140)
summary.config(state='disabled', bg='lightblue')
summary.pack()

# Sentiment
sentiment_label = tk.Label(root, text='Sentiment')
sentiment_label.pack()
sentiment = tk.Text(root, height=1, width=140)
sentiment.config(state='disabled', bg='lightblue')
sentiment.pack()

# URL Entry
url_label = tk.Label(root, text='Enter URL:')
url_label.pack()
url_entry = tk.Entry(root, width=140)
url_entry.pack()

# Language Selection Combobox
lang_label = tk.Label(root, text='Select Language for Translation:')
lang_label.pack()
dest_lang = ttk.Combobox(root, values=list(LANGUAGES.keys()), width=22)
dest_lang.set("en")
dest_lang.pack()

# Summarize Button
btn = tk.Button(root, text='Summarize', command=summarize)
btn.pack()

# Text-to-Speech Button
tts_btn = tk.Button(root, text='Read Article', command=speak)
tts_btn.pack()

# Translate Button
translate_btn = tk.Button(root, text='Translate', command=translate)
translate_btn.pack()

# Clipboard Button
copy_btn = tk.Button(root, text='Copy Summary to Clipboard', command=copy_to_clipboard)
copy_btn.pack()

# Save Button
save_btn = tk.Button(root, text='Save', command=savefile)
save_btn.pack()

# Start the Tkinter event loop
root.mainloop()
